In [1]:
import torch
import pandas as pd
import torch.nn as nn
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split

class GRUNetwork(nn.Module):
    """
    A simple GRU network for sequence prediction.
    """

    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUNetwork, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        return self.fc(gru_out[:, -1, :])  # Return last time step

In [5]:
def load_pamap2_data():
    df = pd.read_parquet('../data/PAMAP2/protocol.parquet')
    # the labels are on the 2º column
    X = df.drop('activityID', axis=1)
    y = df['activityID']
    return X, y

X, y = load_pamap2_data()
X.head()

,timestamp,heartrate,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,handGyro1,...,ankleGyro1,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,ankleOrientation1,ankleOrientation2,ankleOrientation3,ankleOrientation4
0,5.64,NaN,33.0,2.79143,7.55389,-7.06374,2.87553,7.88823,-6.76139,1.01640,...,-0.027148,-0.031190,-0.040897,-47.7695,-2.58701,59.8481,-0.012871,0.747947,-0.079841,0.658813
1,5.65,NaN,33.0,2.86086,7.43814,-7.21626,2.84248,7.63164,-6.85140,1.08269,...,0.012804,-0.036384,-0.014845,-47.7624,-2.81438,60.3407,0.014025,-0.748410,0.079043,-0.658360
2,5.66,100.0,33.0,2.80997,7.09430,-7.52376,2.79373,7.43545,-7.07744,1.14829,...,-0.001695,-0.034529,-0.040074,-47.5393,-2.38909,60.9623,0.012714,-0.749508,0.075286,-0.657577
3,5.67,NaN,33.0,2.77138,6.98077,-7.52396,2.80627,7.29929,-7.24326,1.24888,...,0.055437,-0.011560,0.015434,-47.4246,-2.72082,60.7103,0.012639,-0.749201,0.076030,-0.657843
4,5.68,NaN,33.0,2.76710,7.13119,-7.67848,2.78766,7.11797,-7.48445,1.27178,...,0.017184,-0.034663,0.007239,-47.5412,-2.60153,60.3412,0.012571,-0.749291,0.076346,-0.657705


In [ ]:
def objective_function(hidden_size, num_layers, learning_rate):
    """
    Objective function for Bayesian Optimization.

    This function initializes the GRU network with the given hyperparameters,
    trains it on the provided dataloader, and returns the average loss as
    the evaluation metric.

    Parameters:
    -----------
    hidden_size : float
        The number of hidden units in the GRU layer. This value should be
        between 10 and 100.

    num_layers : float
        The number of GRU layers in the network. This value should be between
        1 and 5.

    learning_rate : float
        The learning rate for the optimizer. This value should be between
        0.0001 and 0.1.

    Returns:
    --------
    float
        The average loss of the model after training.
    """
    
    # Load PAMAP2 dataset
    X, y = load_pamap2_data()
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Set up the LSTM network
    input_size = X.shape[2]  # Number of features
    output_size = 1          # Single output for binary classification

    model = LSTMNetwork(input_size, int(hidden_size), int(num_layers), output_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCEWithLogitsLoss()  # Use for binary classification

    model.train()  # Set model to training mode

    for epoch in range(num_epochs):
        for i in range(len(X_train)):
            optimizer.zero_grad()
            output = model(torch.tensor(X_train[i:i+1], dtype=torch.float32))  # Shape (1, seq_length, input_size)
            loss = criterion(output, torch.tensor(y_train[i:i+1], dtype=torch.float32).view(-1, 1))
            loss.backward()
            optimizer.step()

    # Evaluate AUC-ROC
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        y_pred = model(torch.tensor(X_test, dtype=torch.float32))
        y_pred_prob = torch.sigmoid(y_pred).numpy()  # Apply sigmoid to get probabilities

    auc_roc = roc_auc_score(y_test, y_pred_prob)
    return auc_roc  # Return the AUC-ROC score

# Define the bounds for hyperparameters
pbounds = {
    'hidden_size': (10, 100),  # Example bounds for hidden layer size
    'num_layers': (1, 5),      # Example bounds for number of layers
    'learning_rate': (0.0001, 0.1)  # Example bounds for learning rate
}

# Run Bayesian Optimization
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=5,
    n_iter=25,
)

print(optimizer.max)
